In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import pandas as pd
import numpy as np
from icecream import ic
from tqdm import tqdm
from typing import List

In [3]:
from src.retriever.transformer.splitter import recursive_text_split, DEFAULT_SEPARATORS

In [4]:
df_results = pd.DataFrame(
    columns=["Name", "Year", "Author", "Description", "IsIndexed", "NbPages"]
)
df_results

,Name,Year,Author,Description,IsIndexed,NbPages


In [5]:
doc_path = "../data/raw/province-sud/BaieDesCitrons_barriere_01062023.pdf"

# Data Model

## Pydantic

In [6]:
from pydantic import BaseModel
import uuid

In [7]:
class SubDocument(BaseModel):
    sub_id: uuid.UUID = None
    text: str = None
    embedding: list[float] = None


class Document(BaseModel):
    id: uuid.UUID = None
    author: str = None
    nb_pages: int = None
    nb_sub_docs: int = None
    sub_documents: list[SubDocument] = None
    embedding_model: str = None
    splitter_method: str = None

# Loader

## pdf loader

### Pypdf

In [8]:
from pypdf import PdfReader


def load_pdf(pdf_path: str):
    reader = PdfReader(pdf_path)
    nb_pages = len(reader.pages)
    list_sub_docs = []
    for page in reader.pages:
        list_sub_docs.append(SubDocument(sub_id=uuid.uuid1(), text=page.extract_text()))
    return Document(id=uuid.uuid1(), nb_pages=nb_pages, sub_documents=list_sub_docs)

In [9]:
doc = load_pdf(doc_path)
doc

Document(id=UUID('9d046bbe-76cd-11ee-bd91-acbc32b740c5'), author=None, nb_pages=20, nb_sub_docs=None, sub_documents=[SubDocument(sub_id=UUID('9baafef4-76cd-11ee-bd91-acbc32b740c5'), text='VILLE DE NOUMEA   \nDAODPM Dispositif de barrière anti -requin  à la Baie des Citrons  \nDossier  n°A001.20039.001  Page 1 sur 20 \n Mai 2023  ISO9001  : FDT1_V6/09 -21 \n \n \n \n \n \n \nVille de Nouméa  \nRésumé non technique du d ossier d’A utorisation \nd’Occupa tion du Domaine Public Maritime  \n(DAODPM)  \nEtude d’impact de la m ise en place d’un dispositif de \nbarrière  anti-requin au niveau de la Baie des Citrons  \nMai 2023  \nDEPARTEMENT : Environnement  \nDossiers n° : A001.210 39.001 \n \n \n \n \n \n \n \n \n \n \n \n \nAgence Nouméa \uf0b7 1Bis rue Berthelot, BP 3583, 98846 Nouméa Cedex  \nTél. (687) 28 34 80 \uf0b7 Fax (687) 28 83 44 \uf0b7 secretariat@soproner.nc   \n \nLe système qualité de GINGER SOPRONER est certifié ISO 9001 -2015 par   \n', embedding=None), SubDocument(sub_id=UU

In [10]:
len(doc.sub_documents[0].text)

721

### Split documents

In [11]:
def split_documents(docs: List[Document]) -> List[Document]:
    for doc in docs:
        new_sub_documents = []
        for sub_doc in doc.sub_documents:
            list_txt = recursive_text_split(
                text=sub_doc.text,
                separators=DEFAULT_SEPARATORS,
                keep_separator=False,
                chunk_size=500,
                chunk_overlap=50,
            )
            for text in list_txt:
                new_sub_documents.append(SubDocument(sub_id=uuid.uuid1(), text=text))
        doc.sub_documents = new_sub_documents
        doc.nb_sub_docs = len(new_sub_documents)
        doc.splitter_method = "Recursive"
    return docs

In [12]:
doc_splitted = split_documents([doc])
doc_splitted

[Document(id=UUID('9d046bbe-76cd-11ee-bd91-acbc32b740c5'), author=None, nb_pages=20, nb_sub_docs=106, sub_documents=[SubDocument(sub_id=UUID('a0152730-76cd-11ee-bd91-acbc32b740c5'), text='VILLE DE NOUMEA   \nDAODPM Dispositif de barrière anti -requin  à la Baie des Citrons  \nDossier  n°A001.20039.001  Page 1 sur 20 \n Mai 2023  ISO9001  : FDT1_V6/09 -21 \n \n \n \n \n \n \nVille de Nouméa  \nRésumé non technique du d ossier d’A utorisation \nd’Occupa tion du Domaine Public Maritime  \n(DAODPM)  \nEtude d’impact de la m ise en place d’un dispositif de \nbarrière  anti-requin au niveau de la Baie des Citrons  \nMai 2023  \nDEPARTEMENT : Environnement  \nDossiers n° : A001.210 39.001 \n \n \n \n \n \n \n ', embedding=None), SubDocument(sub_id=UUID('a0152956-76cd-11ee-bd91-acbc32b740c5'), text='Dossiers n° : A001.210 39.001 \n \n \n \n \n \n \n \n \n \n \n \n \nAgence Nouméa \uf0b7 1Bis rue Berthelot, BP 3583, 98846 Nouméa Cedex  \nTél. (687) 28 34 80 \uf0b7 Fax (687) 28 83 44 \uf0b7 secr

### Nougat

# Embeddings

In [36]:
import os
from dotenv import load_dotenv

load_dotenv()


def get_openai_key():
    return os.getenv("OPENAI_API_KEY")

## OpenAI

In [37]:
import openai

openai.api_key = get_openai_key()
OPENAI_MODEL_EMBEDDING = "text-embedding-ada-002"

In [46]:
def get_embedding(list_text, model=OPENAI_MODEL_EMBEDDING):
    list_text = [text.replace("\n", " ") for text in list_text]
    return openai.Embedding.create(input=list_text, model=model)


def get_embedding_values(embedding: dict):
    return embedding["data"]


def get_nb_tokens(embedding: dict):
    return embedding["usage"]["total_tokens"]

In [70]:
from time import sleep

nb_token_used = 0
doc.embedding_model = OPENAI_MODEL_EMBEDDING

for idx_sub_doc in tqdm(range(0, len(doc.sub_documents), 20)):
    results = get_embedding([doc.text for doc in doc.sub_documents[idx_sub_doc:idx_sub_doc+20]], model=OPENAI_MODEL_EMBEDDING)
    for idx, emb in enumerate(results["data"]):
        doc.sub_documents[idx_sub_doc+idx].embedding = emb["embedding"]
    nb_token_used += get_nb_tokens(results)
    sleep(20)

ic(f"{nb_token_used} were used.")

100%|██████████| 6/6 [02:06<00:00, 21.14s/it]
ic| f"{nb_token_used} were used.": '13945 were used.'


'13945 were used.'

### Export embeddings

In [71]:
import pickle

with open(
    "../data/processed/province-sud/BaieDesCitrons_barriere_01062023_2.pickle", "wb"
) as handle:
    pickle.dump(doc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [72]:
with open(
    "../data/processed/province-sud/BaieDesCitrons_barriere_01062023_2.pickle", "rb"
) as handle:
    data = pickle.load(handle)

data

Document(id=UUID('9d046bbe-76cd-11ee-bd91-acbc32b740c5'), author=None, nb_pages=20, nb_sub_docs=106, sub_documents=[SubDocument(sub_id=UUID('a0152730-76cd-11ee-bd91-acbc32b740c5'), text='VILLE DE NOUMEA   \nDAODPM Dispositif de barrière anti -requin  à la Baie des Citrons  \nDossier  n°A001.20039.001  Page 1 sur 20 \n Mai 2023  ISO9001  : FDT1_V6/09 -21 \n \n \n \n \n \n \nVille de Nouméa  \nRésumé non technique du d ossier d’A utorisation \nd’Occupa tion du Domaine Public Maritime  \n(DAODPM)  \nEtude d’impact de la m ise en place d’un dispositif de \nbarrière  anti-requin au niveau de la Baie des Citrons  \nMai 2023  \nDEPARTEMENT : Environnement  \nDossiers n° : A001.210 39.001 \n \n \n \n \n \n \n ', embedding=[0.0006982518825680017, -0.01186123676598072, -0.015948228538036346, -0.018098551779985428, -0.00260347593575716, 0.046314649283885956, -0.03539762273430824, 0.010634449310600758, -0.02362598478794098, -0.015465783886611462, 0.013832366093993187, 0.0037699572276324034, 0.0240

### Similarities

In [91]:
question = "Quelles sont les risques liés à la barrière?"
query_embedding = get_embedding([question])["data"][0]["embedding"]
passage_embedding = [sub_doc.embedding for sub_doc in doc.sub_documents]
similarities = util.dot_score(query_embedding, passage_embedding)
top_indices = np.argsort(similarities)[0][-5:]
list_context = []
for ind in top_indices:
    print(similarities[0][ind])
    list_context.append(doc.sub_documents[ind].text)
list_context

tensor(0.8228)
tensor(0.8244)
tensor(0.8346)
tensor(0.8410)
tensor(0.8487)


["(panneaux / sentier sous -marin  pédagogique ) \n- Seule la baignade sera autorisée dans l'emprise du dispositif  2 3 1 Moyenne  Probable  Moyenne  \nDestruction directe de la faune et de la flore en cas de \ndécrochage de la barrière  3 3 2 Majeure  Possible  Moyenne  - Dimensionnement du dispositif pour répondre aux \ncontraintes du milieu  et résister à des conditions climatiques \nextrêmes  \n- Utilisation de matériaux appropriés  \n- Surveillance périodique de l'état des composants de la ",
 'La compilation des données sur les EIP (Écosystèmes d’Intérêt Patrimonial) et des ERM \n(Espèces Rares ou Menacées) a permis de réaliser une carte des enjeux envir onnementaux \nde la baie induis par la pose de la barrière ( Figure 4). ',
 "- Barrière à larges mailles  \n- Dimensionnement de la barrière adapté aux conditions de \nhoule et de marée  1 3 1 Moyenne  Certaine  Moyenne  \nSubstrat  Risque d'abra sion avec le mouvement de la barrière  et de \nmodification de la nature du substrat

### Generative answer

In [95]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in French:"""

question_prompt = prompt_template.format(context="\n".join(list_context), question=question)

In [102]:
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct",
  prompt=question_prompt,
  max_tokens=1000
)
response

<OpenAIObject text_completion id=cmpl-8FDgWAcNnVbsOQ6NXHuzGxZd99nrK at 0x1484dbcc0> JSON: {
  "id": "cmpl-8FDgWAcNnVbsOQ6NXHuzGxZd99nrK",
  "object": "text_completion",
  "created": 1698638216,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": " Certains risques li\u00e9s \u00e0 la barri\u00e8re pourraient inclure la destruction de la faune et de la flore, l'abrasion et la modification du substrat, ainsi que des risques li\u00e9s \u00e0 la sant\u00e9 et s\u00e9curit\u00e9 des usagers tels qu'une augmentation de la s\u00e9curit\u00e9 face au risque requin, le risque de noyade et l'augmentation de l'accidentologie aux abords de la barri\u00e8re.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 708,
    "completion_tokens": 93,
    "total_tokens": 801
  }
}

In [99]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_template},
        {"role": "user", "content": question}
    ]
)
response

<OpenAIObject chat.completion id=chatcmpl-8FDeh8vryO3Ha8PphjEJpu40zXAG1 at 0x153b365e0> JSON: {
  "id": "chatcmpl-8FDeh8vryO3Ha8PphjEJpu40zXAG1",
  "object": "chat.completion",
  "created": 1698638103,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Je ne sais pas."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 5,
    "total_tokens": 80
  }
}

In [97]:
response

<OpenAIObject text_completion id=cmpl-8FDcKEN8Oypn63iLinDd6HBgPxLUS at 0x14f709130> JSON: {
  "id": "cmpl-8FDcKEN8Oypn63iLinDd6HBgPxLUS",
  "object": "text_completion",
  "created": 1698637956,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": " Les risques li\u00e9s \u00e0 la barri\u00e8re sont la destruction directe de la",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 708,
    "completion_tokens": 16,
    "total_tokens": 724
  }
}

## Sentence Transformer

In [103]:
from sentence_transformers import SentenceTransformer, util

ST_MODEL_EMBEDDING = "multi-qa-MiniLM-L6-cos-v1"

model = SentenceTransformer(ST_MODEL_EMBEDDING)

doc_st = doc.model_copy(deep=True)
doc_st.embedding_model = ST_MODEL_EMBEDDING

for sub_docs in tqdm(doc_st.sub_documents):
    txt_ = sub_docs.text.replace("\n", " ")
    sub_docs.embedding = list(model.encode(txt_))

100%|██████████| 106/106 [00:08<00:00, 13.15it/s]


In [14]:
doc_st.embedding_model

'multi-qa-MiniLM-L6-cos-v1'

In [ ]:
def indices_of_nearest_neighbors_from_distances(distances) -> np.ndarray:
    """Return a list of indices of nearest neighbors from a list of distances."""
    return np.argsort(distances)

In [110]:
query_embedding = model.encode("Quelles sont les risques liés à la barrière?")
passage_embedding = [doc.embedding for doc in doc_st.sub_documents]
similarities = util.dot_score(query_embedding, passage_embedding)
top_indices = indices_of_nearest_neighbors_from_distances(similarities)[0][-5:]
for ind in top_indices:
    print(doc_st.sub_documents[ind].text)

- Barrière à larges mailles  
- Dimensionnement de la barrière adapté aux conditions de 
houle et de marée  1 3 1 Moyenne  Certaine  Moyenne  
Substrat  Risque d'abra sion avec le mouvement de la barrière  et de 
modification de la nature du substrat  1 3 2 Moyenne  Probable  Moyenne  - Fixation régulière et adaptée  ainsi que lest adapté  
- Choix d'un tracé limitant les surfaces sensibles à l'abrasion  
- Mise en place d’un système fixe sur la zone sensible du récif 
VILLE DE NOUMEA   
DAODPM Dispositif de barrière anti -requin  à la Baie des Citrons  
Dossier  n°A001.20039.001  Page 8 sur 20 
 Mai 2023  ISO9001  : FDT1_V6/09 -21 
prévisionnelle de  la phase de  commande, livraison et  travaux de pose de la barrière, hors 
approvisionnement et hors intempéries, est de 5 mois  dont 1,5 mois pour les travaux à 
proprement parler . 
II.2. ENTRETIEN ET SURVEILL ANCE  
Le concept de la barrière est d’être maintenue en pl ace toute l’année même en conditions 
Il retiendra les flotteurs et 

In [106]:
indices_of_nearest_neighbors_from_distances(similarities)[0]

tensor([ 76,  11,   1,   8,  43,  93,  91,  78,  54,  44,  74,   3,  17,  10,
          7, 105,  77,  81,  86,  71,  49,  12,  53,  38,  66,  50,  90,  34,
         84,  42,  46,  87,  57,  82,  62,  15,  63,  95,  59,  75,  48,  88,
         19,  96,  83,  80,  47,  69,  39,  37,   0,  22,  36,  79,  70,  67,
         92,  31, 100,  98,  89,  20,  68, 102,  33,  13,  73,  41,  25,  40,
         64,   5, 104,  85,  45,   9,   2,  58,  35,  55,  14,  28,  23,  60,
         16,  56,  18,  21,  72,  24,  97,  32,  52,  51, 101,   6,  29,  99,
         65,   4,  61,  94,  30,  27, 103,  26])